In [1]:
import os

In [2]:
os.chdir('../')

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    target_column: str

In [34]:
from carPred.constants import *
from carPred.utils.common import read_yaml, create_directories

In [36]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            target_column = schema.name
            
        )

        return model_trainer_config

In [37]:
import pandas as pd
import os
from carPred import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [38]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        RF = RandomForestRegressor(n_estimators=self.config.n_estimators, max_depth=self.config.max_depth, random_state=42)
        RF.fit(train_x, train_y)

        joblib.dump(RF, os.path.join(self.config.root_dir, self.config.model_name))


In [39]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-02-23 11:27:56,259: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-23 11:27:56,261: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-23 11:27:56,264: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-23 11:27:56,265: INFO: common: created directory at: artifacts]
[2024-02-23 11:27:56,266: INFO: common: created directory at: artifacts/model_trainer]


d:\Anaconda3new\envs\car_pred\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
